<a href="https://colab.research.google.com/github/AppleTater/Jinkerson-Lab-Research/blob/master/Specific_Gene_Cumulative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Files that need to be uploaded**


---
* TAGData.csv
* large-lib_full_2016.10.25.csv



# Import libraries

In [1]:
%reload_ext rpy2.ipython

# %%R
# library(ggplot2)

import rpy2.robjects as robjects                          # common rpy2 objects under the abbreviation 'robjects'
from rpy2.robjects.packages import importr                # rpy2 packages under the abbreviation 'importr'
from rpy2.robjects.vectors import FloatVector             # rpy2 for float conversion
import gc                                                 # rpy2 library
R_stats = importr('stats')                                # rpy2 stats

from rpy2.rinterface_lib.sexp import StrSexpVector        # rpy2 interface
from functools import partial                             # rpy2 tools under the abbreviation 'partial'
from rpy2.ipython import html                             # html support
html.html_rdataframe=partial(html.html_rdataframe, table_class="docutils")

from rpy2.robjects.conversion import localconverter       # rpy2 converter under the abbreviation 'localconverter'  

import pandas as pd                                       # these libraries are for redundancy
import rpy2.robjects as ro                                # ↓
from rpy2.robjects.packages import importr                # ↓
from rpy2.robjects import pandas2ri                       # ↓

In [2]:
import pandas as pd                                       # importing common pandas libraries under the abbreviation 'pd' 
import matplotlib.pyplot as plt                           # was for the requested histogram under the abbreviation 'plt' 

import numpy.matlib                                       # to calculate mean and median
import numpy as np                                        # numpy under the abbreviation 'np' 
from numpy import median                                  # from the numpy library, import how to get the median

import scipy.stats as stats                               # libraries used for BH adjustment under the abbreviation 'stats' 
import scipy.special                                      # special libraries from scipy

import re                                                 # more libraries
import sys                                                # more libraries

from IPython.display import (                             # display data, if wanted, as different media file types
    display_pretty, display_html, display_jpeg,
    display_png, display_json, display_latex, display_svg
)

from IPython.display import display                       # more display tools
import itertools                                          # more display tools
import gc                                                 # more display tools

import statsmodels.api as sm                              # stats models under the abbreviation 'sm'
import statsmodels.stats.multitest as smm                 # stats models under the abbreviation 'smm'
import statsmodels.formula.api as smf                     # stats models under the abbreviation 'smf'

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm



# Run
---


*   Given gene name, find data (name → data)
*   Given data, find name(s) (data → name)
*   Change list values, let run






# Given name, find data


In [ ]:
tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv'

cre_name = input("Cre name: ")                                                  # user input for specific cre
cre_name_v5 = ".v5.5"                                                           # store value ".v5.5" as cre_name_v5
solo_name = cre_name                                                            # solo_name stores the inputted cre
cre_name += cre_name_v5                                                         # cre_name strores the cre name with .v5.5 appended

count = float(input("Drop data if the total counts of a gene is less than: "))  # user input for counts
confidence = float(input("Drop data if the confidence level is greater than: "))# user input for confidence level
ratio = float(input("'Hits' are ratios that are greater than: "))               # user input for ratio 
#------------------------------------------------------------------------------------------------------------------------------------

# ↓ Gene specific ↓

# for 'TAG Data.csv'
cre_tag_df = tag[['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads' ]]
cre_tag_only = pd.DataFrame()
cre_tag_only = cre_tag_df.loc[((cre_tag_df['gene'] == cre_name) | (cre_tag_df['gene'] == solo_name))]
# keep only the listed columns

# for 'large-lib_full_2016.10.25.csv'
cre_lib_df = lib[['gene', 'IB', 'confidence_level']]
cre_lib_only = cre_lib_df.loc[((cre_lib_df['gene'] == cre_name) | (cre_lib_df['gene'] == solo_name))]
# keep only the listed columns

merged_cre = cre_tag_only.merge(cre_lib_only, on = "IB")                        # merge data from lib and tag based on their IB
cre_count = merged_cre['gene_x'].value_counts()                                 # store the number of times the specified gene is in merged_cre, stored as cre_count

merged_cre['Ratios: HLG / ALL (Whole)'] = merged_cre['R5-S#11-HLG_WHOLE_normalized_reads'] / merged_cre['R5-S#11-ALL_WHOLE_normalized_reads']       
# calculate HLG / ALL for ratio
whole = merged_cre
#merged_cre = merged_cre.replace([np.inf, -np.inf], np.nan)                     # where there is a value equal to 'inf', it is replaced by 'NaN' 

merged_cre = merged_cre[merged_cre.confidence_level <= confidence]              # restricts merged_cre to rows that have confidence levels greater than inputted confidence

merged_cre = merged_cre.drop(columns = {'gene_y'}) #'R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged_cre = merged_cre.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns and reset index

merged_cre["RatioType"] = np.where(merged_cre["Ratios: HLG / ALL (Whole)"] >= ratio, "Hit", "Non-Hit")
# new column in merged_cre, 'RatioType' to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'
merged_cre = merged_cre.dropna().reset_index(drop = True)    
#merged_cre = merged_cre.reset_index(drop = True)
# drop rows with Nan and reset index

# ↓ Not gene specific ↓

# for 'TAG Data.csv'
tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
# keep only the listed columns
tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
tag_df.reset_index(drop = True)                                                 # reset index

# for 'large-lib_full_2016.10.25.csv'
lib_df = lib[['gene', 'IB', 'confidence_level' ]]
# keep only the listed columns
lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
lib_df = lib_df.reset_index(drop = True)                                        # reset index
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
# calculate HLG / ALL for ratio      
merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns
merged["RatioType"] = np.where(merged["Ratios: HLG / ALL (Whole)"] >= ratio, "Hit", "Non-Hit")
# new column in merged, RatioType to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'
merged = merged[['Gene', 'IB', "Confidence Level", "Ratios: HLG / ALL (Whole)", "RatioType"]]
# reorder merged to match the column order of merged_cre
merged = merged.sort_values('Gene')
# sort the values by the gene name
merged = merged.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
# replace inf by Nan, drop rows that has Nan, reset the index

merged_count_drop = pd.DataFrame(data = merged['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
merged_count_drop = merged['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
# drop rows that has a count that is less than the inputted count restriction
merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
# drop rows that have Nan, sort dataframe by gene name, reset index

clean = merged.merge(merged_count_drop, on = "Gene")                            # merge merged and merged_count_drop based on gene name
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Counts total number of specific gene hits and specific gene non-hits based on ratio cutoff
given = pd.DataFrame(data = merged_cre, columns = ['Gene'])                     # dataframe based on merged_cre, only gene names
given = given['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
given = given.sort_values('Gene').reset_index(drop = True)                      # sort dataframe by gene name, reset index

given['Hit'] = len(merged_cre[merged_cre.RatioType == 'Hit'])                   # count number of 'Hit' in RatioType from merged_cre
given['Non-Hit'] = len(merged_cre[merged_cre.RatioType == "Non-Hit"])           # count number of 'Non-Hit' in RatioType from merged_cre

# Counts of all unique genes, counts of hits and non-hits based on ratio cutoff
counts = pd.DataFrame(data = clean['Gene'], columns = ['Gene'])                 # dataframe based on clean, only gene names
counts = counts['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
counts = counts.sort_values('Gene').reset_index(drop = True)                    # sort dataframe by gene name, reset index

counts['Hit'] = len(clean[clean.RatioType == 'Hit'])                            # count the number of 'Hit' in RatioType from clean
counts['Non-Hit'] = len(clean[clean.RatioType == "Non-Hit"])                    # count the number of 'Non-Hit' in RatioType from clean

counts = counts[counts['Count'] >= count].reset_index(drop = True)              # drops rows that have a count less than the inputted count, rest index
specific = counts.loc[counts['Gene'] == solo_name]                              # specific becomes the row that contains the specified cre name

frames = [given, specific]                                                      # creates an array(?) that are the given and specific dataframes

p_val_calc = pd.concat(frames).drop(columns = {'Count'}).reset_index(drop = True)
# concatenate the dataframes in frames, drop the 'Count' column, reset index
p_val_calc.iloc[1,0] = 'Total'                                                  # change value in the contingency table from the gene name to 'Total'
 # contingency table:                          
        # |            |   Hit   |   Non-Hit   |
        # |————————————|—————————|—————————————|  
        # |  cre_name  |         |             |
        # |————————————|—————————|—————————————|  
        # |    Total   |         |             |
p_val_calc = p_val_calc.set_index('Gene')                                       # make the 'Gene' column to be the index

oddsratio, pvalue = stats.fisher_exact(p_val_calc)                              # calculate p-value
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

display (p_val_calc)
display(merged_cre)                                                             # display dataframe for the specifed gene (Gene, IB, Confidence Level, Ratio, RatioType)
print(cre_count)                                                                # print all isntances of the specified gene count prior to dropping rows
print(pvalue)                                                                   # print p-value


# Given data, find name(s)

In [9]:
tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv' 

count = float(input("Drop data if the total counts of a gene is less than: "))  # user input for counts
confidence = float(input("Drop data if the confidence level is greater than: "))# user input for confidence level
ratio = float(input("'Hits' are ratios that are greater than: "))               # user input for ratio 
bh_num = float(input("Enter the value that all adjusted p-values should be less than (0-1): "))
# user input for the maximum of the adjusted p-value
#------------------------------------------------------------------------------------------------------------------------------------

# for 'TAG Data.csv'
tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
# keep only the listed columns
tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
tag_df.reset_index(drop = True)                                                 # reset index

# for 'large-lib_full_2016.10.25.csv'
lib_df = lib[['gene', 'IB', 'confidence_level' ]]
# keep only the listed columns
lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
# drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
lib_df = lib_df.reset_index(drop = True)                                        # reset index
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
# calculate HLG / ALL for ratio      
merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
# drop listed columns
merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
# rename columns
merged["RatioType"] = np.where(merged["Ratios: HLG / ALL (Whole)"] >= ratio, "Hit", "Non-Hit")
# new column in merged, RatioType to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'
merged = merged[['Gene', 'IB', "Confidence Level", "Ratios: HLG / ALL (Whole)", "RatioType"]]
# reorder merged 
merged = merged.sort_values('Gene')
# sort the values by the gene name
merged = merged.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
# replace inf by Nan, drop rows that has Nan, reset the index

merged_count_drop = pd.DataFrame(data = merged['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
merged_count_drop = merged['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
# count the number of instances of each gene, stored in a new column, 'Count'
merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
# drop rows that has a count that is less than the inputted count restriction
merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
# drop rows that have Nan, sort dataframe by gene name, reset index

clean = merged.merge(merged_count_drop, on = "Gene")                            # merge merged and merged_count_drop based on gene name
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# total counts for all genes based on ratio (Hit vs Non-Hit counts)
total_counts = clean['RatioType'].value_counts()                                # RatioType column is counted hit and non-hit
total_counts = pd.DataFrame(total_counts)                                       # makes into a dataframe
total_counts = total_counts.reset_index()                                       # reset the index
total_counts = total_counts.rename(columns = {'index' : 'Gene', 'RatioType' : 'Hit'})
# rename columns
total_counts['Non-Hit'] = ""                                                    # make empty column
total_counts.iloc[0,0] = 'Total'                                                # replace value in cell (0,0) to 'Total'
total_counts.iloc[0,2] = total_counts.iloc[0,1]                                 # replace value in cell (0,2) with value in (0,1)
total_counts.iloc[0,1] = total_counts.iloc[1,1]                                 # replace value in cell (0,1) with value in (1,1)
total_counts = total_counts.drop([1])                                           # drop last row
total_counts = total_counts.set_index('Gene')                                   # make index of the dataframe to be the 'Gene' column
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

hit_type = clean[['Gene', 'RatioType', 'Count']]                                # new dataframe, takes only 'Gene' 'RatioType', and 'Count' from clean
unique_gene = pd.DataFrame(hit_type.Gene.unique(), columns = {'Gene'})          # new dataframe, with only unique gene names
unique_gene['Hit'] = ""                                                         # empty column in unique_gene
unique_gene['Non-Hit'] = ""                                                     # empty column in unique_gene

pd.options.mode.chained_assignment = None  # default='warn'                     # remove warning messages

# ↓ Benjamini-Hochberg Adjustment of p-values ↓
# calls on a statisical adjustment from an R library:
def FDR_adjust_pvalues(case, N=None, method='BH'):
    """ Adjust a list of p-values for false discovery rate using R's stats::p.adjust function.

    N and method are passed to R_stats.p_adjust:
     - N is the number of comparisons (if left unspecified, defaults to len(pvalue_list), I think)
     - method is the name of the adjustment method to use (inherited from R)

    Note that this MUST be done after all the p-values are already collected, on the full list of p-values at once:
     trying to do it on single p-values, even with adjusted N, will give different results!
    """
    if not method in R_stats.p_adjust_methods:
        raise ValueError("Unknown method %s - method must be one of (%s)!"%(method, ', '.join(R_stats.p_adjust_methods)))
    if N is None:   return R_stats.p_adjust(FloatVector(case), method=method)
    else:           return R_stats.p_adjust(FloatVector(case), method=method, n=N)

super_x = []                                                                    # empty list

for i in range(len(unique_gene)):                                               # for i in the range of the number of rows in unique_genes
  same_gene = hit_type.loc[hit_type.Gene == unique_gene.iloc[i,0]]              # splice hit_type by the values of unique_gene (gene names)
  num_hit = len(same_gene[(same_gene['RatioType'] == 'Hit')])                   # num_hit is the number of times 'Hit' is counted in the RatioType column of same_gene
  num_non_hit = len(same_gene[(same_gene['RatioType'] == 'Non-Hit')])           # num_non_hit is the number of times 'Non-Hit' is counted in the RatioType column of same_gene
  same_gene['Hit'] = num_hit                                                    # new column in same_gene, stores the number of hits
  same_gene['Non-Hit'] = num_non_hit                                            # new column in same_gene, stores the number of non-hits
  same_gene = same_gene.drop_duplicates(subset = 'Gene', keep = "first").drop(columns = {'RatioType', 'Count'}).set_index('Gene')
  # keep only first row, drop columns, make 'Gene' column index
  frames = [same_gene, total_counts]                                            # store same_gene and total_counts as a 2 x 2 matrix
  contingency_table = pd.concat(frames)                                         # make the 2 x 2 matrix into a dataframe with a 2 x 2 dimension
  oddsratio, pvalue = stats.fisher_exact(contingency_table)                     # run the fisher exact test on the contingency table, find p-value
  contingency_table ['p-value'] = pvalue                                        # store the p-value in a new column in contingency_table
  contingency_table = contingency_table.drop(['Total'])                         # drop Total
  
  super_x.append(contingency_table)                                             # grow super_x list


super_x = pd.concat(super_x, axis = 0)                                          # concatenate super_x and make it into a dataframe
super_x["Adjusted p-value"] = FDR_adjust_pvalues(super_x['p-value'])
# run FDR_adjust_pvalues on the p-value column, and store new value in "Adjusted p-value" column
super_x = super_x.loc[super_x["Adjusted p-value"] < bh_num]                     # show genes that have an adjusted p-value that is less than bh_num 

display(super_x, total_counts)                        


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Drop data if the total counts of a gene is less than: 3
Drop data if the confidence level is greater than: 4
'Hits' are ratios that are greater than: 0.19
Enter the value that all adjusted p-values should be less than (0-1): 0.5


,Hit,Non-Hit,p-value,Adjusted p-value
Gene,,,,
Cre01.g000017,0,4,0.311692,1.0
Cre01.g000200,1,2,1.000000,1.0
Cre01.g000300,4,3,0.223976,1.0
Cre01.g000350,1,3,1.000000,1.0
Cre01.g000500,1,2,1.000000,1.0
...,...,...,...,...
Cre36.g759697,4,2,0.091477,1.0
Cre38.g759997,3,9,0.762024,1.0
Cre39.g760097,37,67,0.528795,1.0


,Hit,Non-Hit
Gene,,
Total,6632,13821


# List adjustment and running

In [23]:
# CHANGE LISTS
l_count = [20, 30, 40]
l_confidence = [1]
l_ratio = [0.19]
l_bh_num = [0.5]

In [38]:
tag = pd.read_csv('TAG Data.csv')                                               # read in 'TAG Data.csv'
lib = pd.read_csv('large-lib_full_2016.10.25.csv')                              # read in 'large-lib_full_2016.10.25.csv' 

#------------------------------------------------------------------------------------------------------------------------------------
# start for loop here:
for count in l_count:
  for confidence in l_confidence:
    for ratio in l_ratio:
      for bh_num in l_bh_num:
        # for 'TAG Data.csv'
        tag_df = tag [['gene', 'IB', 'R5-S#11-HLG_WHOLE_normalized_reads' , 'R5-S#11-ALL_WHOLE_normalized_reads']]
        # keep only the listed columns
        tag_df = tag_df.loc[~((tag_df['gene'] == "unknown_chrom") | (tag_df['gene'] == "gene_unknown")),:]
        # drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
        tag_df = tag_df[~tag_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
        tag_df.reset_index(drop = True)                                                 # reset index

        # for 'large-lib_full_2016.10.25.csv'
        lib_df = lib[['gene', 'IB', 'confidence_level' ]]
        # keep only the listed columns
        lib_df = lib_df.loc[~((lib_df['gene'] == "unknown_chrom") | (lib_df['gene'] == "gene_unknown")),:]
        # drop rows that have 'unknown_chrom' or 'gene_unknown' as gene values
        lib_df = lib_df[~lib_df.gene.str.contains (" & ")]                              # drop rows that have an ampersand (sometimes the gene names are 2, joined by an ampersand)
        lib_df = lib_df[lib_df.confidence_level >= confidence]                          # restricts lib_df to rows that have confidence levels greater than inputted confidence
        lib_df = lib_df.reset_index(drop = True)                                        # reset index
        #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        merged = tag_df.merge(lib_df, on = "IB")                                        # merge tag_df to lib_df based on their IB
        merged ['Ratios: HLG / ALL (Whole)'] = merged ['R5-S#11-HLG_WHOLE_normalized_reads'] / merged['R5-S#11-ALL_WHOLE_normalized_reads']
        # calculate HLG / ALL for ratio      
        merged = merged.drop(columns = {'gene_y','R5-S#11-HLG_WHOLE_normalized_reads', 'R5-S#11-ALL_WHOLE_normalized_reads'})
        # drop listed columns
        merged = merged.rename(columns = {'gene_x' : 'Gene', 'confidence_level' : "Confidence Level"}).reset_index(drop = True)
        # rename columns
        merged["RatioType"] = np.where(merged["Ratios: HLG / ALL (Whole)"] >= ratio, "Hit", "Non-Hit")
        # new column in merged, RatioType to compare the ratios to the inputted ratio to determine what is a 'Hit' or a 'Non-Hit'
        merged = merged[['Gene', 'IB', "Confidence Level", "Ratios: HLG / ALL (Whole)", "RatioType"]]
        # reorder merged 
        merged = merged.sort_values('Gene')
        # sort the values by the gene name
        merged = merged.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop = True)
        # replace inf by Nan, drop rows that has Nan, reset the index

        merged_count_drop = pd.DataFrame(data = merged['Gene'], columns = ['Gene'])     # new dataframe, merged_count_drop, gets unique genes
        merged_count_drop = merged['Gene'].value_counts().to_frame().reset_index().rename(columns = {'index' : 'Gene', 'Gene' : 'Count'})
        # count the number of instances of each gene, stored in a new column, 'Count'
        merged_count_drop = merged_count_drop[merged_count_drop['Count'] >= count].reset_index(drop = True)
        # drop rows that has a count that is less than the inputted count restriction
        merged_count_drop  = merged_count_drop.dropna().sort_values('Gene').reset_index(drop = True)
        # drop rows that have Nan, sort dataframe by gene name, reset index

        clean = merged.merge(merged_count_drop, on = "Gene")                            # merge merged and merged_count_drop based on gene name
        #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        # total counts for all genes based on ratio (Hit vs Non-Hit counts)
        total_counts = clean['RatioType'].value_counts()                                # RatioType column is counted hit and non-hit
        total_counts = pd.DataFrame(total_counts)                                       # makes into a dataframe
        total_counts = total_counts.reset_index()                                       # reset the index
        total_counts = total_counts.rename(columns = {'index' : 'Gene', 'RatioType' : 'Hit'})
        # rename columns
        total_counts['Non-Hit'] = ""                                                    # make empty column
        total_counts.iloc[0,0] = 'Total'                                                # replace value in cell (0,0) to 'Total'
        total_counts.iloc[0,2] = total_counts.iloc[0,1]                                 # replace value in cell (0,2) with value in (0,1)
        total_counts.iloc[0,1] = total_counts.iloc[1,1]                                 # replace value in cell (0,1) with value in (1,1)
        total_counts = total_counts.drop([1])                                           # drop last row
        total_counts = total_counts.set_index('Gene')                                   # make index of the dataframe to be the 'Gene' column
        #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        hit_type = clean[['Gene', 'RatioType', 'Count']]                                # new dataframe, takes only 'Gene' 'RatioType', and 'Count' from clean
        unique_gene = pd.DataFrame(hit_type.Gene.unique(), columns = {'Gene'})          # new dataframe, with only unique gene names
        unique_gene['Hit'] = ""                                                         # empty column in unique_gene
        unique_gene['Non-Hit'] = ""                                                     # empty column in unique_gene

        pd.options.mode.chained_assignment = None  # default='warn'                     # remove warning messages

        # ↓ Benjamini-Hochberg Adjustment of p-values ↓
        # calls on a statisical adjustment from an R library:
        def FDR_adjust_pvalues(case, N=None, method='BH'):
            """ Adjust a list of p-values for false discovery rate using R's stats::p.adjust function.

            N and method are passed to R_stats.p_adjust:
            - N is the number of comparisons (if left unspecified, defaults to len(pvalue_list), I think)
            - method is the name of the adjustment method to use (inherited from R)

            Note that this MUST be done after all the p-values are already collected, on the full list of p-values at once:
            trying to do it on single p-values, even with adjusted N, will give different results!
            """
            if not method in R_stats.p_adjust_methods:
                raise ValueError("Unknown method %s - method must be one of (%s)!"%(method, ', '.join(R_stats.p_adjust_methods)))
            if N is None:   return R_stats.p_adjust(FloatVector(case), method=method)
            else:           return R_stats.p_adjust(FloatVector(case), method=method, n=N)

        super_x = []                                                                    # empty list

        for i in range(len(unique_gene)):                                               # for i in the range of the number of rows in unique_genes
          same_gene = hit_type.loc[hit_type.Gene == unique_gene.iloc[i,0]]              # splice hit_type by the values of unique_gene (gene names)
          num_hit = len(same_gene[(same_gene['RatioType'] == 'Hit')])                   # num_hit is the number of times 'Hit' is counted in the RatioType column of same_gene
          num_non_hit = len(same_gene[(same_gene['RatioType'] == 'Non-Hit')])           # num_non_hit is the number of times 'Non-Hit' is counted in the RatioType column of same_gene
          same_gene['Hit'] = num_hit                                                    # new column in same_gene, stores the number of hits
          same_gene['Non-Hit'] = num_non_hit                                            # new column in same_gene, stores the number of non-hits
          same_gene = same_gene.drop_duplicates(subset = 'Gene', keep = "first").drop(columns = {'RatioType', 'Count'}).set_index('Gene')
          # keep only first row, drop columns, make 'Gene' column index
          frames = [same_gene, total_counts]                                            # store same_gene and total_counts as a 2 x 2 matrix
          contingency_table = pd.concat(frames)                                         # make the 2 x 2 matrix into a dataframe with a 2 x 2 dimension
          oddsratio, pvalue = stats.fisher_exact(contingency_table)                     # run the fisher exact test on the contingency table, find p-value
          contingency_table ['p-value'] = pvalue                                        # store the p-value in a new column in contingency_table
          contingency_table = contingency_table.drop(['Total'])                         # drop Total
          
          super_x.append(contingency_table)                                             # grow super_x list


        super_x = pd.concat(super_x, axis = 0)                                          # concatenate super_x and make it into a dataframe
        super_x["Adjusted p-value"] = FDR_adjust_pvalues(super_x['p-value'])
        # run FDR_adjust_pvalues on the p-value column, and store new value in "Adjusted p-value" column
        num_bh_adj = super_x.loc[super_x["Adjusted p-value"] < bh_num].shape[0]                     # show genes that have an adjusted p-value that is less than bh_num  
        print ("Count: ", count, " | ", "Confidence: ", confidence, " | ", "Ratio: ", ratio, " | ", "Adjusted p-value", bh_num, " | ", "Number of significant genes: ", num_bh_adj)
        display(super_x.sort_values("Adjusted p-value"), total_counts)                        


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Count:  20  |  Confidence:  1  |  Ratio:  0.19  |  Adjusted p-value 0.5  |  Number of significant genes:  7


,Hit,Non-Hit,p-value,Adjusted p-value
Gene,,,,
Cre04.g229550,17,10,0.001491,0.321656
Cre13.g589400,14,7,0.001680,0.321656
Cre04.g217935,5,37,0.002824,0.360471
Cre12.g556150,14,9,0.006312,0.402926
Cre06.g284850,14,9,0.006312,0.402926
...,...,...,...,...
Cre06.g264850,8,12,0.480479,1.000000
Cre05.g245352,5,17,0.372526,1.000000
Cre05.g244250,9,14,0.509091,1.000000


,Hit,Non-Hit
Gene,,
Total,5126,10612


Count:  30  |  Confidence:  1  |  Ratio:  0.19  |  Adjusted p-value 0.5  |  Number of significant genes:  0


,Hit,Non-Hit,p-value,Adjusted p-value
Gene,,,,
Cre04.g217935,5,37,0.004257,0.600195
Cre13.g589450,377,673,0.015441,0.828230
Cre04.g217951,150,386,0.041118,0.828230
Cre02.g112300,29,34,0.021765,0.828230
Cre12.g554400,54,76,0.029532,0.828230
...,...,...,...,...
Cre06.g278099,18,38,1.000000,1.000000
Cre06.g278108,13,27,1.000000,1.000000
Cre13.g603176,15,33,1.000000,1.000000


,Hit,Non-Hit
Gene,,
Total,3252,6846


Count:  40  |  Confidence:  1  |  Ratio:  0.19  |  Adjusted p-value 0.5  |  Number of significant genes:  7


,Hit,Non-Hit,p-value,Adjusted p-value
Gene,,,,
Cre04.g217935,5,37,0.004240,0.356166
Cre12.g554400,54,76,0.029716,0.478690
Cre10.g450000,22,24,0.038508,0.478690
Cre13.g589450,377,673,0.021142,0.478690
Cre04.g217951,150,386,0.039891,0.478690
...,...,...,...,...
Cre10.g439350,73,146,0.769842,1.000000
Cre10.g451600,22,40,0.587977,1.000000
Cre10.g455190,13,27,1.000000,1.000000


,Hit,Non-Hit
Gene,,
Total,2654,5557
